# 1. Import Libraries:

In [1]:
base_dir = "/kaggle/input/tomatoleaf/tomato"
train_dir = "/kaggle/input/tomatoleaf/tomato/train"
validation_dir = "/kaggle/input/tomatoleaf/tomato/val"

# 2. Define Image Size and Batch Size:

In [2]:
IMAGE_SIZE = 224 # define the image size for all images (height and Width = 224 X 224)
BATCH_SIZE = 64  # At the time of training 64 images will be there at a time

# 3. Set Folder Paths:

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from glob import glob
from sklearn.model_selection import train_test_split

2024-05-14 15:26:14.098182: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-14 15:26:14.098290: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-14 15:26:14.241185: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 4. Load VGG16 Model:

In [4]:
# IMAGE_SIZE = [224, 224]
vgg = VGG16(weights = 'imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)) 
# include_top = False (where we are defining our own class/label) means we are working at our own dataset where we will have our own label
# here we will have 10 layers (as 10 classes are there)
vgg.output

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


<KerasTensor shape=(None, 7, 7, 512), dtype=float32, sparse=False, name=keras_tensor_18>

# 5. Freeze Convolutional Layers (Optional Fine-Tuning):

In [5]:
# we donot need to train our layer, we will work with pre existing weights
for layer in vgg.layers[:10]:
    layer.trainable = False

for layer in vgg.layers[12:]:
    layer.trainable = True
# We have total 19 layers

# 6. Get the Number of Classes from Folder Names:


In [6]:
# folders will contain the all folder under train folder
# folders = glob("/kaggle/input/tomatoleaf/tomato/train/*")
folders = glob(train_dir + '/*')
print(len(folders))

num_classes = len(folders)
print("Number of classes:", num_classes)

10
Number of classes: 10


# 7. Create the Classification Head:

In [7]:
model = Sequential()
model.add(vgg)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))                   # Dropout layer
model.add(Dense(256, activation='relu'))  # Additional dense layer
model.add(Dropout(0.3))                   # Dropout layer
model.add(Dense(128, activation='relu'))  # Additional dense layer
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))
print(model)

<Sequential name=sequential, built=False>


# 8. Compile the Model (Initial Training):

In [8]:
optimizer = RMSprop(learning_rate=0.0001)  # Consider using a learning rate scheduler
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

## Prepare the data.

# 9. Data Augmentation for Training and Validation:

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split = 0.2  # 20% for validation
)
validation_datagen = ImageDataGenerator(
    rescale=1./255,
)

# 10. Prepare Data Generators:

In [10]:
train_generator = train_datagen.flow_from_directory(
    train_dir, # define directory
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,  # Shuffle the data before splitting
    subset='training',  # Specify that this is for training
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',  # Specify that this is for training
)
test_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    shuffle = False,
    class_mode = 'categorical'
)

Found 8000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


# 11. Early Stopping and Learning Rate Reduction:

In [11]:
early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=2, min_lr=0.0001)
model_checkpoint = ModelCheckpoint('tomato_vgg16_model.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=1)

# 12. Train the Model (Initial):

In [12]:
epoch = 20
history = model.fit(
    train_generator, epochs=epoch, validation_data=validation_generator, callbacks=[early_stopping, reduce_lr, model_checkpoint]
)

Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-05-14 15:26:52.071305: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng15{k5=1,k6=0,k7=1,k10=1} for conv (f32[64,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,3,224,224]{3,2,1,0}, f32[64,3,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0} is taking a while...
2024-05-14 15:26:52.498659: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 100352: 3.87759, exp

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1398 - loss: 2.3047

W0000 00:00:1715700585.998354      69 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
2024-05-14 15:30:19.127314: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 3.89842, expected 3.37692
2024-05-14 15:30:19.127368: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 12: 3.13412, expected 2.61262
2024-05-14 15:30:19.127378: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 13: 3.08377, expected 2.56227
2024-05-14 15:30:19.127386: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 14: 4.1021, expected 3.5806
2024-05-14 15:30:19.127395: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 29: 3.76539, expected 3.24388
2024-05-14 15:30:19.127403: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 30: 4.19841, expected 3.6769
2024-05-14 15:30:19.127411: E external/local_xla/xla/service/gp


Epoch 1: val_loss improved from inf to 1.72074, saving model to tomato_vgg16_model.weights.h5
125/125 ━━━━━━━━━━━━━━━━━━━━ 232s 1s/step - accuracy: 0.1402 - loss: 2.3037 - val_accuracy: 0.4085 - val_loss: 1.7207 - learning_rate: 1.0000e-04
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 880ms/step - accuracy: 0.3604 - loss: 1.7698
Epoch 2: val_loss improved from 1.72074 to 1.15320, saving model to tomato_vgg16_model.weights.h5
125/125 ━━━━━━━━━━━━━━━━━━━━ 146s 1s/step - accuracy: 0.3608 - loss: 1.7690 - val_accuracy: 0.5900 - val_loss: 1.1532 - learning_rate: 1.0000e-04
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 892ms/step - accuracy: 0.5377 - loss: 1.3284
Epoch 3: val_loss improved from 1.15320 to 0.97166, saving model to tomato_vgg16_model.weights.h5
125/125 ━━━━━━━━━━━━━━━━━━━━ 148s 1s/step - accuracy: 0.5380 - loss: 1.3277 - val_accuracy: 0.6540 - val_loss: 0.9717 - learning_rate: 1.0000e-04
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 873ms/step - accuracy: 0.6515 - loss: 1.0192
Epoch 4

# 13. Evaluate the Model after Initial Training:

In [13]:
score = model.evaluate(validation_generator)
print("Test loss (initial):", score[0])
print("Test accuracy (initial):", score[1])

32/32 ━━━━━━━━━━━━━━━━━━━━ 29s 888ms/step - accuracy: 0.9556 - loss: 0.1694
Test loss (initial): 0.23099194467067719
Test accuracy (initial): 0.9440000057220459


# Predict

# 14. Get Classification Report

In [14]:
import numpy as np
from sklearn.metrics import classification_report
class_labels = ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 
                'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite',
                'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus',
                'Tomato___healthy']

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_labels = test_generator.classes
report = classification_report(true_labels, predicted_classes, target_names=class_labels)
print(report)

 2/16 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step 

W0000 00:00:1715702606.441705      66 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


15/16 ━━━━━━━━━━━━━━━━━━━━ 0s 385ms/step

2024-05-14 16:03:33.430708: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 4.65268, expected 3.85372
2024-05-14 16:03:33.430921: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 5.99463, expected 5.19567
2024-05-14 16:03:33.430940: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 6.57741, expected 5.77845
2024-05-14 16:03:33.430950: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 5.98603, expected 5.18707
2024-05-14 16:03:33.430961: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 5.73158, expected 4.93262
2024-05-14 16:03:33.430971: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 6.82611, expected 6.02715
2024-05-14 16:03:33.430981: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 5.87791, expected 5.07895
2024-05-14 16:03:33.430991: E external/local_xla/xla/se

16/16 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step
                                               precision    recall  f1-score   support

                      Tomato___Bacterial_spot       0.94      0.93      0.93       100
                        Tomato___Early_blight       0.95      0.78      0.86       100
                         Tomato___Late_blight       0.88      0.97      0.92       100
                           Tomato___Leaf_Mold       1.00      0.95      0.97       100
                  Tomato___Septoria_leaf_spot       0.83      0.96      0.89       100
Tomato___Spider_mites Two-spotted_spider_mite       0.91      0.90      0.90       100
                         Tomato___Target_Spot       0.83      0.90      0.86       100
       Tomato___Tomato_Yellow_Leaf_Curl_Virus       1.00      0.95      0.97       100
                 Tomato___Tomato_mosaic_virus       0.97      1.00      0.99       100
                             Tomato___healthy       0.92      0.85      0.89       100

  

# 15. Update the weights of the model

In [15]:
model.load_weights('/kaggle/working/tomato_vgg16_model.weights.h5')

# Test an image for prediction

In [16]:
import numpy as np
import os
from keras.preprocessing import image
from keras.models import load_model

image_path = "/kaggle/input/tomatoleaf/tomato/val/Tomato___Septoria_leaf_spot/0a25f893-1b5f-4845-baa1-f68ac03d96ac___Matt.S_CG 7863.JPG"

# Extract the actual class label from the image path
actual_class = os.path.basename(os.path.dirname(image_path))

img_pred = image.load_img(image_path, target_size=(224,224))
img_pred = image.img_to_array(img_pred)
img_pred = np.expand_dims(img_pred, axis=0) # adding a dimension

# Define the list of class labels
class_labels = ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 
                'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite',
                'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus',
                'Tomato___healthy']
# passing the array to the model 
# vgg_model = load_model('/kaggle/working/vgg_model_tomato.h5')
# reslt = vgg_model.predict(img_pred) # return a 2d array
reslt = model.predict(img_pred)
print(reslt)

# Get the predicted class index
predicted_class_index = np.argmax(reslt[0])

# Print the predicted class
predicted_class = class_labels[predicted_class_index]
print("Predicted class:", predicted_class)

# Print the actual class
print("Actual class:", actual_class)

2024-05-14 16:03:54.262400: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 4.42855, expected 3.64542
2024-05-14 16:03:54.262457: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 5.87036, expected 5.08723
2024-05-14 16:03:54.262467: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 6.04003, expected 5.2569
2024-05-14 16:03:54.262475: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 6.22034, expected 5.43722
2024-05-14 16:03:54.262483: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 4.90026, expected 4.11714
2024-05-14 16:03:54.262490: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 5.21068, expected 4.42756
2024-05-14 16:03:54.262498: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 5.92061, expected 5.13748
2024-05-14 16:03:54.262506: E external/local_xla/xla/ser

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
Predicted class: Tomato___Late_blight
Actual class: Tomato___Septoria_leaf_spot
